In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#set path
path = '/media/onno/Volume/GFS_T850/'

Calculate GSS for every single vent on lead days 1, 3, 5, 7 and 9

In [14]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24),
]#Ukraine
seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

#Choose Forecast model: GFS or ERA5RF
fcst_modelz = ['ERA5RF']
for fcst_model in fcst_modelz:
    for lat_1,lat_0,lon_0,lon_1 in coordinatez:
        Path("/media/onno/Volume/GFS_T850/anom_files/lon_{}_{}_lat_{}_{}/"\
             .format(lon_0,lon_1,lat_1,lat_0)).mkdir(parents=True, exist_ok=True)
#         file_fcst = file_dic['T850_grid'][fcst_model].format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_fcst = 'era5rf_fldmean_t850_0-240h_24hourly_2x2nh_jan79-dec19_lon_{}_{}_lat_{}_{}_ALL_NEW.nc'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_rean = file_dic['T850_grid']['ERA5'].format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_clim_p90 = 'era5_mars_t850_79-19_24hourly_90p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_clim_p50 = 'era5_mars_t850_79-19_24hourly_50p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_clim_p10 = 'era5_mars_t850_79-19_24hourly_10p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        fcst = xr.open_dataset(path+file_fcst,decode_times=False).squeeze()
        #set fcst time data to datetime object
        init_time_fcst = pd.Timestamp('1979-01-01')
        fcst['time'] = [pd.Timedelta(i-fcst.time.values[0],'hours')+init_time_fcst for i in fcst.time.values]
        fcst['lead'] = fcst['lead']//24
        #group forecast by lead time and take daily means
        fcst = fcst.sel(lead=slice(0,9)).groupby('lead').mean()
        rean = xr.open_dataset(path+file_rean).squeeze()
        if int(rean.time[0].dt.hour)!=0:
            rean = rean.assign_coords({'time':rean.time.values - pd.Timedelta(int(rean.time[0].dt.hour),'h')})
        #load percentile climatology. For convenience I set the data to all days in the leap year 2016. That has no further effect on the data
        clim_p90 = xr.open_dataset(path+file_clim_p90).squeeze()
        clim_p90['time']=pd.date_range('2016-01-01',"2016-12-31")
        clim_p50 = xr.open_dataset(path+file_clim_p50).squeeze()
        clim_p50['time']=pd.date_range('2016-01-01',"2016-12-31")
        clim_p10 = xr.open_dataset(path+file_clim_p10).squeeze()
        clim_p10['time']=pd.date_range('2016-01-01',"2016-12-31")
        #load all heat extremes
        file_pers_hw = 'dates/pw_lon_{}_{}_lat_{}_{}_new.npy'.format(lon_0,lon_1,lat_1,lat_0)
        file_pers_cw = 'dates/pc_lon_{}_{}_lat_{}_{}_new.npy'.format(lon_0,lon_1,lat_1,lat_0)
        file_short_hw = 'short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_short_cw = 'short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_independent_short_hw = 'independent_short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        file_independent_short_cw = 'independent_short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1-2,lat_1-2,lat_0)
        pers_hw = np.load(path+file_pers_hw,allow_pickle=True)
        pers_cw = np.load(path+file_pers_cw,allow_pickle=True)
        #short_hw = np.load(path+file_short_hw)
        #short_cw = np.load(path+file_short_cw)
        #independent_short_hw = np.load(path+file_independent_short_hw)
        #independent_short_cw = np.load(path+file_independent_short_cw)
        #temp_extremez = [pers_hw,short_hw,pers_cw,short_cw]
        temp_extremez = [pers_hw,pers_cw]
        columnz = ['persistent_hw','persistent_cw']
        index = pd.date_range(pd.Timestamp('1979-01-01'),pd.Timestamp('2019-12-31'))
        lead_dayz = [3,5,7]
        for lead_day in lead_dayz:
            df_GSS = pd.DataFrame(index=index,columns=columnz,dtype=float)
            df_HK = pd.DataFrame(index=index,columns=columnz,dtype=float)
            for j,temp_extreme in enumerate(temp_extremez):
                column = columnz[j]
                df_contingency = pd.DataFrame(columns=['hits','miss','false_alarm','non_event'])
                for i,date in enumerate(temp_extreme[:,0]):
#                     if (j==1)and(independent_short_hw[i]==False):
#                         continue
#                     if (j==3)and(independent_short_cw[i]==False):
#                         continue
                    date = pd.Timestamp(date)
                    if date<pd.Timestamp('1979-01-01')+pd.Timedelta(lead_day,'days'):
                        continue
                    begin_date = date - pd.Timedelta(lead_day,'days')
                    if np.isin(begin_date,[pd.Timestamp('2014-10-29'),
                                           pd.Timestamp('2014-10-30'),
                                           pd.Timestamp('2014-10-31')]):
                        continue
                    end_date = date + pd.Timedelta(9-lead_day,'days')
                    rean_sub = rean.sel(time=slice(begin_date,end_date)).load()
                    fcst_sub = fcst.sel(time=begin_date).load()

                    timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
                    clim_p50_sub = clim_p50.sel(time=timez).load()
                    if j==0:
                        clim_p90_sub = clim_p90.sel(time=timez).load()
                        rean_sub_anom = rean_sub.t - clim_p90_sub.t.values
                        fcst_sub_anom = fcst_sub.t - clim_p90_sub.t.values
                        assert((rean_sub_anom.sel(time=date)>0).all())
                        rean_sub_anom.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/rean_{}_{:02d}_{:02d}_lead_day_{}_pw_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        fcst_sub_anom.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/fcst_{}_{:02d}_{:02d}_lead_day_{}_pw_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        rean_sub_anom_median = rean_sub.t - clim_p50_sub.t.values
                        fcst_sub_anom_median = fcst_sub.t - clim_p50_sub.t.values
                        rean_sub_anom_median.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/rean_{}_{:02d}_{:02d}_lead_day_{}_pw_50p_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        fcst_sub_anom_median.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/fcst_{}_{:02d}_{:02d}_lead_day_{}_pw_50p_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        rean_bool = rean_sub.t>clim_p90_sub.t.values
                        fcst_bool = fcst_sub.t>clim_p90_sub.t.values
                    else:
                        clim_p10_sub = clim_p10.sel(time=timez).load()
                        rean_sub_anom = rean_sub.t - clim_p10_sub.t.values
                        fcst_sub_anom = fcst_sub.t - clim_p10_sub.t.values
                        assert((rean_sub_anom.sel(time=date)<0).all())
                        rean_sub_anom.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/rean_{}_{:02d}_{:02d}_lead_day_{}_cw_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        fcst_sub_anom.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/fcst_{}_{:02d}_{:02d}_lead_day_{}_cw_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        rean_sub_anom_median = rean_sub.t - clim_p50_sub.t.values
                        fcst_sub_anom_median = fcst_sub.t - clim_p50_sub.t.values
                        rean_sub_anom_median.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/rean_{}_{:02d}_{:02d}_lead_day_{}_cw_50p_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        fcst_sub_anom_median.to_netcdf(path+'anom_files/lon_{}_{}_lat_{}_{}/fcst_{}_{:02d}_{:02d}_lead_day_{}_cw_50p_new.nc'.format(lon_0,lon_1,lat_1,lat_0,
                                                                                                    date.year,date.month,date.day,lead_day))
                        rean_bool = rean_sub.t<clim_p10_sub.t.values
                        fcst_bool = fcst_sub.t<clim_p10_sub.t.values
                        
                    hits = (rean_bool & fcst_bool.values).astype(np.int64).sum()
                    miss = ((~fcst_bool)&(rean_bool.values)).astype(np.int64).sum()
                    false_alarm = ((fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()
                    non_event = ((~fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()            
                    GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
                    HK = my_tools.hansen_kuiper_discrimant(hits,miss,false_alarm,non_event)
                    df_GSS.loc[date][column]=GSS
                    df_HK.loc[date][column]=HK
#                     fig,ax = plt.subplots(figsize=(12,12))
#                     ax.plot(rean_sub.time,fcst_sub.t,label='fcst',marker='o')
#                     ax.plot(rean_sub.time,rean_sub.t,label='rean',marker='o')
#                     ax.plot(rean_sub.time,clim_p90_sub.t,color='r')
#                     ax.legend()
#                     ax.set_title('GSS = {:.2f} HK = {:.2f}'.format(float(GSS),float(HK)))
#                     fig.savefig(path+'GSS/figures/ind_events/{}_{:02d}_{:02d}_warm.png'.format(date.year,
#                                                                                          date.month,
#                                                                                          date.day))
            df_GSS = df_GSS.dropna(how='all')
            df_HK = df_HK.dropna(how='all')

            df_GSS.to_csv(path+'GSS/GSS_lead_day_{}_lon_{}_{}_lat_{}_{}_{}_new.csv'.format(lead_day,lon_0,lon_1,
                                                                             lat_0,lat_1,fcst_model))
            df_HK.to_csv(path+'HK/HK_lead_day_{}_lon_{}_{}_lat_{}_{}_{}_new.csv'.format(lead_day,lon_0,lon_1,
                                                                             lat_0,lat_1,fcst_model))
            index_season = ['DJF','MAM','JJA','SON']
            df_season_mean_GSS = pd.DataFrame(index=index_season,columns=['persistent_hw','persistent_cw'])
            df_season_mean_HK = pd.DataFrame(index=index_season,columns=['persistent_hw','persistent_cw'])
            for season in index_season:
                df_season_GSS = df_GSS[np.isin(df_GSS.index.month,seasonz[season])].mean()
                df_season_mean_GSS.loc[season]=df_season_GSS.values
                df_season_HK = df_HK[np.isin(df_HK.index.month,seasonz[season])].mean()
                df_season_mean_HK.loc[season]=df_season_HK.values
#             df_month_mean = df.groupby(df.index.month).mean()
#             df_month_mean.index = month_to_season_lu
#             df_season_mean = df_month_mean.groupby(df_month_mean.index).mean()
            
#             JJA,MAM = a.iloc[1].copy(),a.iloc[2].copy()
#             df_season_mean.iloc[1]=MAM;df_season_mean.iloc[2]=JJA
#             df_season_mean.index=index_month
            df_season_mean_GSS.to_csv(path+'GSS/Monthly_mean_GSS_lead_day_{}_lon_{}_{}_lat_{}_{}_{}_new.csv'.format(lead_day,lon_0,lon_1,
                                                                                       lat_0,lat_1,fcst_model))
            df_season_mean_HK.to_csv(path+'HK/Monthly_mean_HK_lead_day_{}_lon_{}_{}_lat_{}_{}_{}_new.csv'.format(lead_day,lon_0,lon_1,
                                                                                       lat_0,lat_1,fcst_model))
        #     df_month = df.groupby(df.index.month,dropna=True).mean()
        #     df_month.to_csv(path+'GSS/Gilbert_Skill_Score_Monthly_Mean_lead_day_{}.csv'.format(lead_day))
        df_count = df_GSS.groupby(df_GSS.index.month,dropna=True).count()
        df_count.to_csv(path+'GSS/Count_lon_{}_{}_lat_{}_{}_new.csv'.format(lon_0,lon_1,lat_0,lat_1))
        df_season_count = pd.DataFrame(index=index_season,columns=['persistent_hw','persistent_cw'])
        for season in index_season:
            df_count_season = df_count[np.isin(df_count.index,seasonz[season])].sum()
            df_season_count.loc[season]=df_count_season.values
        df_season_count.to_csv(path+'GSS/Count_season_lon_{}_{}_lat_{}_{}_new.csv'.format(lon_0,lon_1,lat_0,lat_1))
#     df.to_csv(path+'GSS/GSS_Temp_Extremes_Seasonally_Grouped_lead_day_{}.csv'.format(lead_day))

    

General monthly Gilbert Skill Scores

In [9]:
plt.close('all')

In [10]:
#put climatologies in list to loop over them
climz = [clim_p90,clim_p10]
#set and select lead day. Value must range between 0 and 9
lead_dayz = [1,3,5,7,9]
#make reanalysis and forecast of equal length
df_warm = pd.DataFrame(index=np.arange(1,13),columns=['day_1','day_3','day_5','day_7','day_9'])
df_cold = pd.DataFrame(index=np.arange(1,13),columns=['day_1','day_3','day_5','day_7','day_9'])
#loop over heat and cold extremes
columnz = ['day_1','day_3','day_5','day_7','day_9']
for lead_day in lead_dayz:
    fcst_lead = fcst.sel(lead=lead_day)
    #convert time of forecast to initial time + forecast lead time
    fcst_lead['time'] = fcst.time + pd.Timedelta(lead_day,'days')
    rean_lead = rean.sel(time=slice(pd.Timestamp('1984-12-{:02d}'.format(lead_day+1)),
                             pd.Timestamp('2019-11-30')))
    fcst_lead = fcst_lead.sel(time=slice(pd.Timestamp('1984-12-{:02d}'.format(lead_day+1)),
                             pd.Timestamp('2019-11-30')))
    for j,clim in enumerate(climz):
        column = columnz[j]
        #calculate anomalies from climatoogy: see my_tools.py for more documentation
        rean_anom = my_tools.calculate_anomalies(rean_lead,clim)    
        fcst_anom = my_tools.calculate_anomalies(fcst_lead,clim)

        if j ==0:
            #convert anomalies to boolean array. If temperature is extreme than value is converted to True
            rean_bool = rean_anom.t>0
            fcst_bool = fcst_anom.t>0
            #fill in contingency table and group all values by month
            hits = (rean_bool & fcst_bool).astype(np.int64).groupby('time.month').sum()
            miss = ((~fcst_bool)&(rean_bool)).astype(np.int64).groupby('time.month').sum()
            false_alarm = ((fcst_bool)&(~rean_bool)).astype(np.int64).groupby('time.month').sum()
            non_event = ((~fcst_bool)&(~rean_bool)).astype(np.int64).groupby('time.month').sum()
            #caluclate Gilbert Skill Score. See my_tools.py for more documentation
            GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
            #Write result to csv file
            df_warm['day_{}'.format(lead_day)]=pd.Series(GSS,index=np.arange(1,13))
        else:
            #convert anomalies to boolean array. If temperature is extreme than value is converted to True
            rean_bool = rean_anom.t<0
            fcst_bool = fcst_anom.t<0
            #fill in contingency table and group all values by month
            hits = (rean_bool & fcst_bool).astype(np.int64).groupby('time.month').sum()
            miss = ((~fcst_bool)&(rean_bool)).astype(np.int64).groupby('time.month').sum()
            false_alarm = ((fcst_bool)&(~rean_bool)).astype(np.int64).groupby('time.month').sum()
            non_event = ((~fcst_bool)&(~rean_bool)).astype(np.int64).groupby('time.month').sum()
            #caluclate Gilbert Skill Score. See my_tools.py for more documentation
            GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
            #Write result to csv file
            df_cold['day_{}'.format(lead_day)]=pd.Series(GSS,index=np.arange(1,13))
df_warm.to_csv(path+'GSS/GSS_general_warm_lon_{}_{}_lat_{}_{}_{}.csv'.format(lon_0,lon_1,lat_0,lat_1,fcst_model))
df_cold.to_csv(path+'GSS/GSS_general_cold_lon_{}_{}_lat_{}_{}_{}.csv'.format(lon_0,lon_1,lat_0,lat_1,fcst_model))

Specific heat waves

In [13]:
#load all heat extremes
file_pers_hw = 'persistent_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_1,lat_0)
file_pers_cw = 'persistent_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_1,lat_0)
file_short_hw = 'short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_1,lat_0)
file_short_cw = 'short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_1,lat_0)
pers_hw = np.load(path+file_pers_hw)
pers_cw = np.load(path+file_pers_cw)
short_hw = np.load(path+file_short_hw)
short_cw = np.load(path+file_short_cw)
temp_extremez = [pers_hw,short_hw,pers_cw,short_cw]

columnz = ['persistent_hw','short_hw','persistent_cw','short_cw',]
index = pd.date_range(pd.Timestamp('1984-12-01'),pd.Timestamp('2019-11-30'))
df = pd.DataFrame(index=index,columns=columnz,dtype=float)
lead_dayz = [1,3,5,7,9]
for lead_day in lead_dayz:
    for j,temp_extreme in enumerate(temp_extremez):
        column = columnz[j]
        for i,date in enumerate(temp_extreme[:,0]):
            if date<pd.Timestamp('1984-12-01')+pd.Timedelta(lead_day,'days'):
                continue
            begin_date = date - pd.Timedelta(lead_day,'days')
            end_date = date + pd.Timedelta(9-lead_day,'days')
            rean_sub = rean.sel(time=slice(begin_date,end_date)).load()
            fcst_sub = fcst.sel(time=begin_date).load()
            timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
            if j<2:
                clim_p90_sub = clim_p90.sel(time=timez).load()
                rean_bool = rean_sub.t>clim_p90_sub.t.values
                fcst_bool = fcst_sub.t>clim_p90_sub.t.values
            else:
                clim_p10_sub = clim_p10.sel(time=timez).load()
                rean_bool = rean_sub.t<clim_p10_sub.t.values
                fcst_bool = fcst_sub.t<clim_p10_sub.t.values
            hits = (rean_bool & fcst_bool.values).astype(np.int64).sum()
            miss = ((~fcst_bool)&(rean_bool.values)).astype(np.int64).sum()
            false_alarm = ((fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()
            non_event = ((~fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()            
            GSS = my_tools.gilbert_skill_score(hits,miss,false_alarm,non_event)
            df.loc[date][column]=GSS

    df = df.dropna(how='all')
    df.to_csv(path+'GSS/Gilbert_Skill_Score_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'.format(lead_day,lon_0,lon_1,
                                                                                           lat_0,lat_1,fcst_model))
#     df_month = df.groupby(df.index.month,dropna=True).mean()
#     df_month.to_csv(path+'GSS/Gilbert_Skill_Score_Monthly_Mean_lead_day_{}.csv'.format(lead_day))
df_count = df.groupby(df.index.month,dropna=True).count()
df_count.to_csv(path+'GSS/Temp_extremes_count_lon_{}_{}_lat_{}_{}.csv'.format(lon_0,lon_1,lat_0,lat_1))

    

Calculate GSS based on monthly grouped contingency table

In [15]:
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw',]
index = pd.date_range(pd.Timestamp('1984-12-01'),pd.Timestamp('2019-11-30'))
df = pd.DataFrame(index=np.arange(1,13),columns=columnz,dtype=float)
lead_dayz = [1,3,5,7,9]
for lead_day in lead_dayz:
    for j,temp_extreme in enumerate(temp_extremez):
        column = columnz[j]
        df_contingency = pd.DataFrame(columns=['hits','miss','false_alarm','non_event'])
        for i,date in enumerate(temp_extreme[:,0]):
            if date<pd.Timestamp('1984-12-01')+pd.Timedelta(lead_day,'days'):
                continue
            begin_date = date - pd.Timedelta(lead_day,'days')
            end_date = date + pd.Timedelta(9-lead_day,'days')
            rean_sub = rean.sel(time=slice(begin_date,end_date)).load()
            fcst_sub = fcst.sel(time=begin_date).load()
            timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
            if j<2:
                clim_p90_sub = clim_p90.sel(time=timez).load()
                rean_bool = rean_sub.t>clim_p90_sub.t.values
                fcst_bool = fcst_sub.t>clim_p90_sub.t.values
            else:
                clim_p10_sub = clim_p10.sel(time=timez).load()
                rean_bool = rean_sub.t<clim_p10_sub.t.values
                fcst_bool = fcst_sub.t<clim_p10_sub.t.values
            hits = (rean_bool & fcst_bool.values).astype(np.int64).sum()
            miss = ((~fcst_bool)&(rean_bool.values)).astype(np.int64).sum()
            false_alarm = ((fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()
            non_event = ((~fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()    
            df_contingency.loc[date] = [int(hits),int(miss),int(false_alarm),int(non_event)]

        df_contingency_month = df_contingency.groupby(df_contingency.index.month).sum()
        GSS = my_tools.gilbert_skill_score_dataframe(df_contingency_month)
        df[column]=GSS
    df.to_csv(path+'GSS/GSS_Temp_Extremes_Monthly_Grouped_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'.format(lead_day,lon_0,lon_1,
                                                                                                         lat_0,lat_1,fcst_model))

    

Calculate GSS based on seasonally grouped contingency table

In [40]:
columnz = ['persistent_hw','persistent_cw']
index = pd.date_range(pd.Timestamp('1984-12-01'),pd.Timestamp('2019-11-30'))
df = pd.DataFrame(index=['DJF','JJA','MAM','SON'],columns=columnz,dtype=float)
month_to_season_lu = np.array([
    None,
    'DJF', 'DJF',
    'MAM', 'MAM', 'MAM',
    'JJA', 'JJA', 'JJA',
    'SON', 'SON', 'SON',
    'DJF'
])
lead_dayz = [1,3,5,7,9]
for lead_day in lead_dayz:
    print(' Lead Day is {}'.format(lead_day))
    for j,temp_extreme in enumerate(temp_extremez):
        column = columnz[j]
        df_contingency = pd.DataFrame(columns=['hits','miss','false_alarm','non_event'])
        for i,date in enumerate(temp_extreme[:,0]):
            date = pd.Timestamp(date)
            if date<pd.Timestamp('1984-12-01')+pd.Timedelta(lead_day,'days'):
                continue
            begin_date = date - pd.Timedelta(lead_day,'days')
            end_date = date + pd.Timedelta(9-lead_day,'days')
            rean_sub = rean.sel(time=slice(begin_date,end_date)).load()
            fcst_sub = fcst.sel(time=begin_date).load()
            timez = [pd.Timestamp('2016-{:02d}-{:02d}'.format(i.month,i.day)) for i in pd.date_range(begin_date,end_date)]
            if j==0:
                clim_p90_sub = clim_p90.sel(time=timez).load()
                rean_bool = rean_sub.t>clim_p90_sub.t.values
                fcst_bool = fcst_sub.t>clim_p90_sub.t.values
            else:
                clim_p10_sub = clim_p10.sel(time=timez).load()
                rean_bool = rean_sub.t<clim_p10_sub.t.values
                fcst_bool = fcst_sub.t<clim_p10_sub.t.values
            hits = (rean_bool & fcst_bool.values).astype(np.int64).sum()
            miss = ((~fcst_bool)&(rean_bool.values)).astype(np.int64).sum()
            false_alarm = ((fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()
            non_event = ((~fcst_bool)&(~rean_bool.values)).astype(np.int64).sum()    
            df_contingency.loc[date] = [int(hits),int(miss),int(false_alarm),int(non_event)]
        if (lead_day==9)&(j==0):
            sys.exit()

        df_contingency_month = df_contingency.groupby(month_to_season_lu[df_contingency.index.month]).sum()
        GSS = my_tools.gilbert_skill_score_dataframe(df_contingency_month)
        df[column]=GSS
    sys.exit()
    print(df)
#     df.to_csv(path+'GSS/GSS_Temp_Extremes_Seasonally_Grouped_lead_day_{}.csv'.format(lead_day))

    

 Lead Day is 1


SystemExit: 

/home/onno/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [50]:
lead_dayz = [1,3,5,7,9]
file = 'GSS/Gilbert_Skill_Score_lead_day_{}.csv'
month_to_season_lu = np.array([
    None,
    'DJF', 'DJF',
    'MAM', 'MAM', 'MAM',
    'JJA', 'JJA', 'JJA',
    'SON', 'SON', 'SON',
    'DJF'
])
for lead_day in lead_dayz:
    df = pd.read_csv(path+file.format(lead_day),index_col=0)
    df.index = pd.to_datetime(df.index)
    df_seasonally = df.groupby(month_to_season_lu[df.index.month]).mean()
    sys.exit()
    

SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
